In [ ]:
from random import randint
import json
import numpy as np
import pandas as pd

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
def coin_flip():
  return randint(0,1)

In [ ]:
def opposite(pos_or_comp):
  if pos_or_comp == 1:
    return 0
  if pos_or_comp == 0:
    return 1
  if pos_or_comp == 'G':
    return 'A'
  if pos_or_comp == 'A':
    return 'G'

In [ ]:
path = '/content/drive/MyDrive/Senior Design/'
data_path = path + "eval/google_generations.json"

In [ ]:
valid_i = []

with open(path + "eval/valid_data_indexes.txt", "r") as f:
  lines = f.readlines()
  for line in lines:
    valid_i.append(line.strip("\n"))

len(valid_i)

138

In [ ]:
google_gen = {}
aTA_gen = {}

with open(path + "eval/google_generations.json", "r", encoding='utf-8') as f:
    google_gen = json.load(f)

with open(path + "eval/rt_generations.json", "r", encoding='utf-8') as f:
    aTA_gen = json.load(f)

In [ ]:
google_gen

{'10268': {'actual:': "It's unclear from your example how exactly you intend to predict if all you have are pairs of terms. If you're assuming that you have all possible correct and incorrect pairs, then you can simply use your data as a hash table where each incorrect term is the key of a correct term. Otherwise, assuming you have all the correct terms but you want to be able to handle slightly changed versions of the incorrect terms. You can define a similarity measure (function) and use that to find the closest incorrect term, in order to return the correct term for it. If you don't have all the correct words, then it's a problem. You should look for a vocabulary of 'legal' terms (or names in your case) and use that as your dictionary. There are some libraries in python that can do that for you, but as far as I know these are all based on regular dictionaries, like the ones used by LibreOffice for example (PyEnchant) is one example for that. If you're looking for a dictionary for na

In [ ]:
aTA_gen

{'10268': {'actual:': "It's unclear from your example how exactly you intend to predict if all you have are pairs of terms. If you're assuming that you have all possible correct and incorrect pairs, then you can simply use your data as a hash table where each incorrect term is the key of a correct term. Otherwise, assuming you have all the correct terms but you want to be able to handle slightly changed versions of the incorrect terms. You can define a similarity measure ( function ) and use that to find the closest incorrect term, in order to return the correct term for it. If you don't have all the correct words, then it's a problem. You should look for a vocabulary of'legal'terms ( or names in your case ) and use that as your dictionary. There are some libraries in python that can do that for you, but as far as I know these are all based on regular dictionaries, like the ones used by LibreOffice for example ( PyEnchant ) is one example for that. If you're looking for a dictionary fo

In [ ]:
eval_csv = []
flip_comps = []

for ind in valid_i:
  ques = aTA_gen[ind]["question:"]
  aTAPos = coin_flip()
  otherPos = opposite(aTAPos)
  
  answers = ["", ""]
  answers[aTAPos] = aTA_gen[ind]["gen:"]

  if ind in google_gen:
    comp = 'G' if coin_flip() == 1 else 'A'
    
    if comp == 'G':
      answers[otherPos] = google_gen[ind]["gen:"]
    else:
      answers[otherPos] = aTA_gen[ind]["actual:"]
  else:
    comp = 'A'
    answers[otherPos] = aTA_gen[ind]["actual:"]

  flip_comps.append(opposite(comp))
  eval_form = [ind+"_1", ques, answers[0], answers[1], aTAPos, comp]
  eval_csv.append(eval_form)

In [ ]:
for i, ind in enumerate(valid_i):
  comp = flip_comps[i]
  ques = aTA_gen[ind]["question:"]
  aTAPos = coin_flip()
  otherPos = opposite(aTAPos)

  answers = ["", ""]
  answers[aTAPos] = aTA_gen[ind]["gen:"]

  if ind in google_gen:  
    if comp == 'G':
      answers[otherPos] = google_gen[ind]["gen:"]
    else:
      answers[otherPos] = aTA_gen[ind]["actual:"]
  else:
    comp = 'A'
    answers[otherPos] = aTA_gen[ind]["actual:"]

  eval_form = [ind+"_2", ques, answers[0], answers[1], aTAPos, comp]
  eval_csv.append(eval_form)

In [ ]:
eval_csv_np = np.array(eval_csv)

In [ ]:
len(eval_csv_np)

276

In [ ]:
eval_csv_df = pd.DataFrame(eval_csv_np, 
                           columns = 
                           ['id','Question','Answer 1', 'Answer 2',
                            'aTA Position', 'Compare To']
                           )

In [ ]:
len(eval_csv_df)

276

In [ ]:
eval_csv_df.head()

,id,Question,Answer 1,Answer 2,aTA Position,Compare To
0,1517_1,How to customise cost function in Scikit learn...,Xma10.... then i suspect that you have somethi...,The test area just has to have at least a slig...,0,G
1,3596_1,number of parameters for convolution layers In...,"Actually it's $ 49C * C $, the first $ C $ is ...",Xyper indeed a relatively also used in this co...,1,A
2,3454_1,Loss function to maximize sum of targets I hav...,"Define $ Z _ i = 1 [ Y _ i 0 ] $, i. e., $ Z _...","I go historical answer, $ i $ in $ m $? in fea...",1,A
3,4198_1,Deep Learning Binary Text Classification I am ...,"In a broader sense, what you're referring to h...",P = then you are interested in this previous e...,1,A
4,5292_1,Implementation of reliable rule learning I wan...,I just read a review of their paper and the di...,Xyperparameter gocatenation is a supervised le...,1,G


In [ ]:
csv_file = path + "eval/evaluation_questions.csv"
eval_csv_df.to_csv(csv_file, sep=',', encoding='utf-8')